# Serving (TensorFlow 1.4)

## 1. Overview

- Một cách đơn giản, để serve 1 model TF, ta cần:
    1. Save model đã train (load từ ckpt) thành SavedModel object.
    2. Build TF Serving `tensorflow_model_server` (binary này dùng để chạy server serving) bằng bazel hoặc install pip-package.
    3. Chạy server với đường dẫn đến SavedModel (có thể enable batching).
    4. Khi có model mới, copy SavedModel mới vào thư mục trên => server sẽ tự động reload model mới.

<img src="serving_architecture.svg">

- TF Serving bao gồm các thành phần sau:
    - **Servable**: là 1 model hoặc một phần model (có thể không phải model TensorFlow). Thông thường thì servable sẽ bao gồm SavedModelBundle (chứa Session) và look-up table. Mỗi servable sẽ có 1 version (số int tăng dần) và nhiều version có thể được chạy cùng lúc (client có thể request 1 version cụ thể). 1 list các version của 1 servable được gọi là **servable stream**.
    - **Loader**: thực hiện load, unload servable. Loader cơ bản nhất nhận đường dẫn đến 1 SavedModel và load, unload model. Loader được dùng để thống nhất API load và unload servable bất kỳ.
    - **Source**: với mỗi servable stream, source tạo ra 1 loader tương ứng để load, unload servable. Source là thành phần poll 1 folder để xem khi nào có version mới.
    - **Manager**: quản lý life-cycle của servable: khi nào cần load, unload hoặc serve. Khi có version mới, source tạo loader cho version này, loader đánh dấu model là ready to load (gọi là **aspired version**) và thông báo cho manager. Manager sẽ quyết định có load và serve version mới này hay không và có unload version cũ hay không. Hiện tại có 2 **version policy** được implement:
        - Availability preserving: load version mới trước rồi mới unload version cũ => luôn có 1 version được chạy nhưng x2 resource.
        - Resource preserving: unload version cũ rồi load version mới.

- TF Serving đã implement những thành phần này để ta có thể serve 1 model với những tính năng cơ bản. Để serve những model phức tạp hoặc cần quy tắc serving đặc biệt, ta có thể extend thêm:
    - Loader: implement loader đọc từ cloud, từ database, ...
    - Source: lưu state của các servable để share lẫn nhau.
    - Batching: cơ chế batching phức tạp hơn hoặc optimize hơn cho model cần serve.
    - Version policy: cơ chế load, unload phức tạp hơn.

## 2. Serving non-Estimator model

### Export SavedModel

- SavedModel là định dạng mới của TF để lưu model (thay cho *SessionBundle*). SavedModel lưu các thành phần sau:
    - Một list các graph (mỗi graph có một list các tag để phân biệt).
    - Giá trị của variable để dùng chung cho tất cả các graph (ta có thể strip device ra khỏi graph).
    - Input, output của những API để serving và loại API đó (gọi là **Signature**, khi lưu dưới dạng protobuf message thì được gọi là **SignatureDefs**).
    - Các asset như dictionary.
    - Các extra-asset khác mà user muốn lưu kèm.

- TF cung cấp SavedModelBuilder để export SavedModel:
    - Meta graph đầu tiên của SavedModel phải được lưu kèm với variable.
    - Những meta graph tiếp theo thì không lưu variable (dùng chung).
    - Mỗi meta graph phải đính kèm một số tag (như training, serving, gpu, ...) để phân biệt chúng và cho biết chức năng từng graph.

In [ ]:
export_dir = ""

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:
    # Build first meta graph
    # ...

    # Add to SavedModel
    builder.add_meta_graph_and_variables(sess,
                                         [tf.saved_model.tag_constants.TRAINING],
                                         signature_def_map=foo_signatures,
                                         assets_collection=foo_assets)

with tf.Session(graph=tf.Graph()) as sess:
    # Build another meta graph
    builder.add_meta_graph(["bar-tag", "baz-tag"])

builder.save()

- TF định nghĩa sẵn một số tag và tên của SignatureDefs thường dùng:
    - `tf.saved_model.tag_constants`: SERVING, TRAINING, GPU, TPU.
    - `tf.saved_model.signature_constants`: CLASSIFY_INPUTS, CLASSIFY_METHOD_NAME, CLASSIFY_OUTPUT_CLASSES, ...

- Mỗi meta-graph trong 1 SavedModel bao gồm nhiều **Signature**. Mỗi meta-graph sẽ có một mapping (dict *signature_def_map*) từ tên của signature (client dùng tên này để request) đến signature đó. Luôn có 1 signature default với tên *serving_default* được sử dụng khi client không request tên 1 signature cụ thể nào.


- TF định nghĩa sẵn 3 loại signature kèm với tên input và output:
    1. Classify (`CLASSIFY_METHOD_NAME`): nhận *input* (`CLASSIFY_INPUTS`) output *classes* (`CLASSIFY_OUTPUT_CLASSES`) và *scores* (`CLASSIFY_OUTPUT_SCORES`) cho bài toán classification.
    2. Regress (`REGRESS_METHOD_NAME`): nhận *input* (`REGRESS_INPUTS`) output giá trị kết quả (`REGRESS_OUTPUTS`) cho bài toán regression.
    3. Predict (`PREDICT_METHOD_NAME`): tổng quát nhất, nhận input (tên user tự đặt) và output kết quả (tên user tự đặt).

    
- TF cung cấp hàm để build SignatureDefs: `tf.saved_model.signature_def_utils.build_signature_def`:
    - inputs: mapping từ tên input đến TensorInfo object (dùng `tf.saved_model.utils_impl.build_tensor_info` để build).
    - outputs: tương tự inputs.
    - method_name: loại API (1 trong 3 loại trên).

In [ ]:
# Example of a classify signature:

signature_def: {
  key  : "my_classification_signature"  # Signature name
  value: {
    inputs: {
      key  : "inputs"
      value: {
        name: "tf_example:0"
        dtype: DT_STRING
        tensor_shape: ...
      }
    }
    outputs: {
      key  : "classes"  # Name of an output
      value: {
        name: "index_to_string:0"  # Mapping to a Tensor in the graph
        dtype: DT_STRING
        tensor_shape: ...
      }
    }
    outputs: {
      key  : "scores"  # Name of another output
      value: {
        name: "TopKV2:0"
        dtype: DT_FLOAT
        tensor_shape: ...
      }
    }
    method_name: "tensorflow/serving/classify"  # Signature type
  }
}

#### Note:
- Classify và Regress luôn phải nhận input là string chứa deserialized tf.Example protobuf nên model khi serve phải parse example này ra Tensor => không được flexible như Predict.

### Run ModelServer

- Để serve model vừa export:
    1. Build `tensorflow_model_server` bằng bazel (có thể build GPU enabled) hoặc install package `tensorflow-model-server`.
    2. Run server: `tensorflow_model_server --model_name=<model-name>  --port=9000 --model_base_path=<path-to-SavedModel>`

### Client

- Sau khi build TF Serving bằng bazel, ta có thể copy những file proto được auto generated để build request phía client (gồm các file `classification_pb2.py`, `get_model_metadata_pb2.py`, `inference_pb2.py`, `input_pb2.py`, `model_pb2.py`, `predict_pb2.py`, `prediction_service_pb2.py`, `regression_pb2.py`).
- Tuy nhiên, hiện tại client request phải gửi lên protobuf message của Tensor nên vẫn phải dùng TF để tạo cho dễ. Ta có thể copy hàm `make_tensor_proto` và các dependency ra dùng riêng để phía client không phải install Tf.

In [ ]:
# Create connection
channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

# Build request
request = predict_pb2.PredictRequest()
request.model_spec.name = "<model-name>"
request.model_spec.signature_name = "<signature-to-request>"
request.inputs["image"].CopyFrom(tf.contrib.util.make_tensor_proto([data], dtype=tf.string))

# Make request
result = stub.Predict(request, 10.0)  # 10 seconds timeout

## 3. Serving Estimator model

### Input function cho model lúc serving

- Estimator hỗ trợ export model đã train thành SavedModel bằng hàm `estimator.export_savedmodel`. Ta cần cung cấp `serving_input_receiver_fn` để tiền xử lý dữ liệu nhận được trong lúc serving và trả về *feature* cho *model_fn*.

- Estimator chỉ hỗ trợ export SavedModel với signature thuộc loại **Predict**. Vì vậy `serving_input_receiver_fn` sẽ nhận Tensor (thay vì tf.Example). Hàm này cần trả về `ServingInputReceiver` - một  cấu trúc dữ liệu lưu mapping giữa Tensor nhận được từ Serving đến input của model_fn.

In [ ]:
def decode_image_string_tensor(encoded_image_string_tensor):
    """Decode an a string of bytes represent an image Tensor."""
    image_tensor = tf.image.decode_image(encoded_image_string_tensor,
                                         channels=3)
    image_tensor.set_shape((None, None, 3))
    return image_tensor

def serving_input_receiver_fn():
    """Parse and preprocess data for serving model."""
    encoded_image_string_tensor = tf.placeholder(
        dtype=tf.string, shape=[None], name='encoded_image_string_tensor')
    receiver_tensors = {
        'image': encoded_image_string_tensor
    }
    images = tf.map_fn(decode_image_string_tensor, encoded_image_string_tensor,
                       back_prop=False, dtype=tf.uint8)
    images = tf.map_fn(lambda image: preprocess_image(image, is_training=False),
                       images, back_prop=False, dtype=tf.float32)

    features = {
        'image': images
    }
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

- Ở ví dụ trên, Serving sẽ gửi cho ta 1 Tensor có tên là **image** thuộc dạng string (lưu JPEG encoded string của một batch các ảnh). Sau đó ta thực hiện decode ảnh và preprocess từng ảnh (dùng `tf.map_fn` để apply 1 function song song cho 1 batch).

- Cuối cùng, ta tạo và return `ServingInputReceiver` với các tham số:
    - `receiver_tensors`: mapping giữa tên Tensor mà client sẽ gửi và placeholder lưu giá trị để return cho model_fn.
    - `features`: dict lưu Tensor để gửi cho model_fn.

##### Note: 
- Nếu không cần tiền xử lý gì đặc biệt thì ta có thể dùng hàm `build_raw_serving_input_receiver_fn` hoặc `build_parsing_serving_input_receiver_fn` để tạo serving_input_receiver_fn.

### Định nghĩa output sẽ serve của model

- `tf.estimator.EstimatorSpec` nhận tham số **export_outputs** để định nghĩa những output của model cho quá trình serving (lưu ý, khi serving thì mode của Estimator sẽ là `tf.estimator.ModeKeys.PREDICT`).

- Tham số này là một dict với key là tên output trong signature và value là ExportOutput object (`ClassificationOutput`, `RegressionOutput` hoặc `PredictOutput`). Lưu ý là signature này vẫn thuộc loại **Predict**, các object output trên chỉ hỗ trợ việc tạo output cho tiện (như classification sẽ tạo 2 giá trị output *classes* và *scores*).

- Nếu *export_outputs* chỉ có 1 phần tử, TF sẽ tự tạo signature *serving_default* với output tương tự phần tử ta cung cấp. Nếu không có, ta phải tự tạo signature này.

- Tất cả signature chỉ khác nhau phần output. Phần input luôn giống nhau là kết quả lấy từ `serving_input_receiver_fn`.

In [ ]:
# Example of export_outputs when creating EsimatorSpec

export_outputs = {
    "scores": tf.estimator.export.PredictOutput({
        "logits": logits,
        "classes": classes
    })
}

tf.estimator.EstimatorSpec(mode, predictions=predictions,
                           loss=total_loss, train_op=train_op,
                           eval_metric_ops=eval_metric_ops,
                           scaffold=scaffold,
                           export_outputs=export_outputs)

### Export Estimator model

- Với input và output được định nghĩa như trên, ta tạo Estimator với *model_dir* dẫn đến checkpoint của model đã train và gọi hàm `export_savedmodel` để tạo SavedModel:

In [ ]:
def serving_input_receiver_fn():
    """Parse and preprocess data for serving model."""
    # Parse and preprocess
    # ...
    
    # Return input data
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)


def model_fn(features, labels, mode, params, config):
    """Model function for Estimator."""
    if mode == tf.estimator.ModeKeys.PREDICT:
        # Build model and output for serving
        # ...

        # Create output data
        export_outputs = {
            "scores": tf.estimator.export.PredictOutput({
                "logits": logits
            })
        }
        
    return tf.estimator.EstimatorSpec(mode, predictions=predictions,
                                      loss=total_loss, train_op=train_op,
                                      eval_metric_ops=eval_metric_ops,
                                      scaffold=scaffold,
                                      export_outputs=export_outputs)
    
    
# Create Estimator and load weights from checkpoint
estimator = tf.estimator.Estimator(model_fn, model_dir=checkpoint_dir)

# Export SavedModel
estimator.export_savedmodel(output_dir, serving_input_receiver_fn)

## 4. Advance: C++ API